In [10]:
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import math
from time import time

In [11]:
t=time()
URL = " https://www.amazon.com/Arm-Hammer-Booster-Purifying-Waters/dp/B08T7X7XYX?pd_rd_w=UPYP1&content-id=amzn1.sym.80bfd2e5-c81d-48d5-ae85-c500e2214af9&pf_rd_p=80bfd2e5-c81d-48d5-ae85-c500e2214af9&pf_rd_r=P3BJ0XMXPK4K68KFG564&pd_rd_wg=a6oPV&pd_rd_r=9a304aff-d87b-4135-b186-63dca4d76e6c&pd_rd_i=B08T7X7XYX&psc=1&ref_=pd_bap_d_csi_pd_ys_c_rfy_rp_crs_6_t"
HEADERS = {
    "method": "GET",

    "accept-encoding": "gzip, deflate, br",

    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36",
    
}


response = requests.get(URL, headers=HEADERS)
#print(response.content)
# with open("myfile.html", "w") as file1:
#     # Writing data to a file
#     file1.write(str(response.content))

soup = BeautifulSoup(response.content, "html.parser")


see_all_reviews_link = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})["href"]


URL_ALL_REVIEWS = "https://www.amazon.com/" + see_all_reviews_link


response = requests.get(URL_ALL_REVIEWS, headers=HEADERS)

soup = BeautifulSoup(response.content, "html.parser")


# Find the element that contains the ratings with reviews and extract the text
ratings_with_reviews_elem = soup.find("div", {"id": "filter-info-section"})
ratings_with_reviews_text = ratings_with_reviews_elem.get_text().strip()

#get just the reviews amount
reviews_count = re.search(r'(\d+,*\d*)\s+with reviews', ratings_with_reviews_text).group(1)

data = []


def scrape_reviews(page_url):
    response = requests.get(page_url, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
    reviews = soup.find_all('div', {'data-hook': 'review'})
    page_data = []
    for review in reviews:
        profile_name = review.find("span", {"class": "a-profile-name"}).get_text()
        title_elem = review.find("a", {"data-hook": "review-title"})
        title = title_elem.get_text().strip() if title_elem else ""
        body = review.find("span", {"data-hook": "review-body"}).get_text().strip()
        stars_elem = review.find("i", {"data-hook": "review-star-rating"})
        stars = float(stars_elem.span.get_text().split()[0]) if stars_elem and stars_elem.span else None
        if stars==None:
            stars_elem = review.find("i", {"data-hook": "cmps-review-star-rating"})
            stars = float(stars_elem.span.get_text().split()[0]) if stars_elem and stars_elem.span else None
        page_data.append([profile_name, title, body, stars])

    return page_data

with ThreadPoolExecutor() as executor:
    page_num=math.ceil(float(reviews_count.replace(",",""))/10)
    pages = [f"{URL_ALL_REVIEWS}+&pageNumber={i+1}" for i in range(page_num)]
    results = executor.map(scrape_reviews, pages)
    for result in results:
        data += result

In [12]:
df = pd.DataFrame(data, columns=['Name', 'Title', 'Body', 'Stars'])
df.head()

,Name,Title,Body,Stars
0,Parker S.,Cleaned my walls with this,I love this mixed in my mop bucket with warm w...,5.0
1,Karol Aguilar,good,"good alternative to other brands, but the scen...",5.0
2,ANIMALGIRL81,They smell so good.,Definitely make my laundry smell great! Will n...,5.0
3,Melissa Y Wood,Value,Smells great and decent price.,5.0
4,Foxylady,Terrific product to drive away bad smells,Great little laundry deodorant. Just enough to...,5.0


In [13]:
print(time()-t)

12.897475242614746


In [14]:
len(df)

806

In [16]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

In [17]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

In [18]:
import openai
import pandas as pd

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()


# Define function to generate summary
def generate_summary(all_text) -> str:
    texts = text_splitter.split_text(all_text)
    docs = [Document(page_content=t) for t in texts[:3]]
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary=chain.run(docs)
    return summary

# Print summary
def chunks_summary(text_chunks):
    # Generate summaries for each text chunk and combine them
    text_chunks=chunk_data(text_chunks,15000) #16122 #15800
    #print(len(text_chunks))
    summaries = []
    for chunk in text_chunks:
        summary = generate_summary(chunk)
        #print(f"Chunk len: {len(chunk)}, Summary len: {len(summary)}")
        summaries.append(summary)
    
    if len(summaries)!=1:
        return chunks_summary(summaries)
    else:
        return summaries[0]
    # combined_summary = " ".join(summaries)
    # overall_summary=generate_summary(combined_summary)


def chunk_data(data, max_len) -> list: 

    res = []
    c_chuck = ""

    for review in data:
        
        if len(c_chuck+review)>max_len:
            if c_chuck:
                res.append(c_chuck)
                c_chuck = ""

        if len(review) > max_len:
            while len(review) > max_len:
                res.append(review[:max_len])
                review = review[max_len:]
            res.append(review)
            c_chuck = ""

        else:
            c_chuck+=review

    if c_chuck:
        res.append(c_chuck)

    return res



summary_of_product=chunks_summary(''.join(df['Body'].to_list()))

In [19]:
print(summary_of_product[1:])

Arm & Hammer Clean Scentsations In-Wash Scent Booster Tropical Paradise is an economical and effective laundry scent booster with a pleasant scent that lasts. It is a great value for the price and is easy to use, but some users have found that the scent does not last long. It is also good for removing odors from work clothes and is gentle on sensitive skin, but some customers have found the scent too strong or not long-lasting enough.


In [21]:
df.head()

,Name,Title,Body,Stars
0,Parker S.,Cleaned my walls with this,I love this mixed in my mop bucket with warm w...,5.0
1,Karol Aguilar,good,"good alternative to other brands, but the scen...",5.0
2,ANIMALGIRL81,They smell so good.,Definitely make my laundry smell great! Will n...,5.0
3,Melissa Y Wood,Value,Smells great and decent price.,5.0
4,Foxylady,Terrific product to drive away bad smells,Great little laundry deodorant. Just enough to...,5.0


In [22]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [23]:
docs_service = build('drive', 'v3', credentials=creds)
file_metadata = {
    'name': 'My Document',
    'parents': ['1lBmUcDPDAiMqyHt3lBIZSvpOkPN5L8Hc'],
    'mimeType': 'application/vnd.google-apps.document'
}
document = docs_service.files().create(body=file_metadata).execute()
document_id=document['id']

In [24]:
# Build the Google Docs API client
docs_service = build('docs', 'v1', credentials=creds)


# Create the table
requests = [{
    'insertTable': {
        'rows': 1,
        'columns':4,
        'endOfSegmentLocation': {}
    }
}]

try:
    result = docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
except HttpError as e:
    print(e)

In [25]:
from gdoctableapppy import gdoctableapp
table_data=[]
table_data.append(df.columns.tolist())
# loop over the rows
for _, row in df.iterrows():
    table_data.append([row['Name'], row['Title'], row['Body'],row['Stars']])

table_data2 = [i[::-1] for i in table_data]

resource = {
    "oauth2": creds,
    "documentId": document_id,
    "tableIndex": 0,
    "values": table_data2
}

res = gdoctableapp.SetValues(resource)

In [26]:
drive_service = build('drive', 'v3', credentials=creds)

# Set up permission metadata
permission_metadata = {
    'role': 'reader',
    'type': 'anyone',
}

# Create the permission
permission = drive_service.permissions().create(
    fileId=document_id,
    body=permission_metadata,
    fields='id'
).execute()

# Get the permission ID
permission_id = permission['id']

# Generate the link
link = f'https://drive.google.com/file/d/{document_id}/view?usp=sharing'
print(link)

https://drive.google.com/file/d/1Ie-QcR3UtIuZje6H8mTWciYciUDMtCa26c6rXQpBJTg/view?usp=sharing
